In [324]:
# All imports here
import os
import librosa
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import svm
import sklearn
import keras
from keras.utils import np_utils
from sklearn.linear_model import LinearRegression
#pip install libav-tools

In [101]:
def create_df(filePath, label):
    fileList = os.listdir(filePath)
    df_obj = pd.DataFrame(fileList)
    # Adding label original to the dataframe to represent original
    df_obj['label'] = label
    df_obj = df_obj.rename(columns={0:'file'})
    df_obj['path'] = filePath
    return df_obj

In [102]:
# Create lists of the files
#fileList1 = os.listdir('recordings/original-recordings')
#df_original = pd.DataFrame(fileList1)
# Adding label original to the dataframe to represent original
#df_original['label'] = 'original'
# Renaming the column
#df_original = df_original.rename(columns={0:'file'})
#df_original

In [103]:
filePaths = ['recordings/original-recordings', 'recordings/speaker-1-desktop-speakers', 'recordings/speaker-2-soundbox', 'recordings/speaker-3-LG_soundbar', 'recordings/speaker-4-Phillips-tv']

pathsAndLabels = {'original': 'recordings/original-recordings', 'desktop': 'recordings/speaker-1-desktop-speakers',
                 'soundbox': 'recordings/speaker-2-soundbox', 'soundbar': 'recordings/speaker-3-LG_soundbar', 'tv': 'recordings/speaker-4-Phillips-tv'}

#filePaths = ['wav-recordings/original_recordings']
#pathsAndLabels = {'original': 'wav-recordings/original_recordings'}

In [104]:
dataFrames = [] #Empty list for storing the datatframes

for label, path in pathsAndLabels.items(): # Loop for iterating through the paths and labels and creating the dataframes
    dataFrames.append(create_df(path, label))
    print(label, path)

for i in range(len(dataFrames)):
    print(dataFrames[i])
#print(dataFrames[5])

original recordings/original-recordings
desktop recordings/speaker-1-desktop-speakers
soundbox recordings/speaker-2-soundbox
soundbar recordings/speaker-3-LG_soundbar
tv recordings/speaker-4-Phillips-tv
                    file     label                            path
0    Innspilling (1).m4a  original  recordings/original-recordings
1   Innspilling (10).m4a  original  recordings/original-recordings
2   Innspilling (11).m4a  original  recordings/original-recordings
3   Innspilling (12).m4a  original  recordings/original-recordings
4   Innspilling (13).m4a  original  recordings/original-recordings
5   Innspilling (14).m4a  original  recordings/original-recordings
6   Innspilling (15).m4a  original  recordings/original-recordings
7   Innspilling (16).m4a  original  recordings/original-recordings
8   Innspilling (17).m4a  original  recordings/original-recordings
9   Innspilling (18).m4a  original  recordings/original-recordings
10  Innspilling (19).m4a  original  recordings/original-reco

In [105]:
# Concatenating all the dataframes into a single one
df = pd.concat(dataFrames, ignore_index=True)

# Loop for renaming the files based on label (This is just done for keeping easier track of dataframes)
#for i in range(len(df)):
    #if df['label'][i] == 'original':
    #    df['file'][i] =  str(0) + df['file'][i]
    #elif df['label'][i] == 'desktop':
    #    df['file'][i] =  str(1) + df['file'][i]
    #elif df['label'][i] == 'soundbox':
    #    df['file'][i] =  str(2) + df['file'][i]
    #elif df['label'][i] == 'soundbar':
    #    df['file'][i] =  str(3) + df['file'][i]
    #else:
    #    df['file'][i] =  str(4) + df['file'][i]
#    df['file'][i] =  str(i) + df['file'][i]
    #print(df['path'][i])

print(df[40:100])

                    file     label                                   path
40  Innspilling (46).m4a  original         recordings/original-recordings
41  Innspilling (47).m4a  original         recordings/original-recordings
42  Innspilling (48).m4a  original         recordings/original-recordings
43  Innspilling (49).m4a  original         recordings/original-recordings
44   Innspilling (5).m4a  original         recordings/original-recordings
45  Innspilling (50).m4a  original         recordings/original-recordings
46   Innspilling (6).m4a  original         recordings/original-recordings
47   Innspilling (7).m4a  original         recordings/original-recordings
48   Innspilling (8).m4a  original         recordings/original-recordings
49   Innspilling (9).m4a  original         recordings/original-recordings
50   Innspilling (1).m4a   desktop  recordings/speaker-1-desktop-speakers
51  Innspilling (10).m4a   desktop  recordings/speaker-1-desktop-speakers
52  Innspilling (11).m4a   desktop  re

In [106]:
#df_copy = df.copy()
#print(df_copy[:60])

# Splitting into test and train data. First randomizing the dataframe containing every sample
df = df.sample(frac=1).reset_index(drop=True)

# Creating a training dataset with 70% of the data
train_df = df.sample(frac=0.7)

# Taking the remaining after the 70 split and putting them into the test datatset
test_df = df.drop(train_df.index)

print(len(test_df))
print(len(train_df))
print(train_df[:60])
print(test_df[:60])


75
175
                     file     label                                   path
87   Innspilling (36).m4a  soundbox          recordings/speaker-2-soundbox
64   Innspilling (26).m4a  original         recordings/original-recordings
144  Innspilling (25).m4a  soundbox          recordings/speaker-2-soundbox
212  Innspilling (44).m4a  soundbox          recordings/speaker-2-soundbox
244  Innspilling (48).m4a  original         recordings/original-recordings
10   Innspilling (17).m4a  soundbox          recordings/speaker-2-soundbox
247   Innspilling (2).m4a  original         recordings/original-recordings
72   Innspilling (21).m4a   desktop  recordings/speaker-1-desktop-speakers
132  Innspilling (37).m4a        tv       recordings/speaker-4-Phillips-tv
239  Innspilling (23).m4a   desktop  recordings/speaker-1-desktop-speakers
205  Innspilling (29).m4a   desktop  recordings/speaker-1-desktop-speakers
120  Innspilling (15).m4a  soundbar       recordings/speaker-3-LG_soundbar
70   Innspilling (

In [107]:
# Checking that the splits have a reasonably balanced amount of different labels
train_df['label'].value_counts(normalize=True)

soundbox    0.228571
desktop     0.211429
original    0.188571
tv          0.188571
soundbar    0.182857
Name: label, dtype: float64

In [108]:
test_df['label'].value_counts(normalize=True)

soundbar    0.240000
tv          0.226667
original    0.226667
desktop     0.173333
soundbox    0.133333
Name: label, dtype: float64

In [109]:
# The splits have about equal amounts and we decide to leave it at this

In [156]:
# Function for extracting the mffcs which we will use as features for the task
# Most of this function is inspired by and based upon https://www.analyticsvidhya.com/blog/2017/08/audio-voice-processing-deep-learning/ in nr # in our appendix

def feature_extract(row):
    # Getting the full path
    #fileName = os.path.abspath(row.path)
    fileName = row.path + '/' + row.file
    print('fileName: ', fileName)
    #print(row.path)

    # Loading the file as floating point time series
    X, sample_rate = librosa.load(fileName, res_type='kaiser_fast')

    # Generate MFCCs from the time series
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)

    feature = mfccs
    label = row.label

    return [feature, label]
    #return [fileName]

tempTrain = train_df.apply(feature_extract, axis=1) # Our new df holding the data with extracted features
print(tempTrain[:10])
#tempTrain.columns = ['feature', 'label']

print("TESTDATA")

tempTest = test_df.apply(feature_extract, axis=1)
print(tempTest[:10])

fileName:  recordings/speaker-2-soundbox/Innspilling (36).m4a
C:\ProgramData\Anaconda3\lib\site-packages\librosa\core\audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
fileName:  recordings/original-recordings/Innspilling (26).m4a
fileName:  recordings/speaker-2-soundbox/Innspilling (25).m4a
fileName:  recordings/speaker-2-soundbox/Innspilling (44).m4a
fileName:  recordings/original-recordings/Innspilling (48).m4a
fileName:  recordings/speaker-2-soundbox/Innspilling (17).m4a
fileName:  recordings/original-recordings/Innspilling (2).m4a
fileName:  recordings/speaker-1-desktop-speakers/Innspilling (21).m4a
fileName:  recordings/speaker-4-Phillips-tv/Innspilling (37).m4a
fileName:  recordings/speaker-1-desktop-speakers/Innspilling (23).m4a
fileName:  recordings/speaker-1-desktop-speakers/Innspilling (29).m4a
fileName:  recordings/speaker-3-LG_soundbar/Innspilling (15).m4a
fileName:  recordings/speaker-

In [336]:
#print(tempTrain[:10])
#print(tempTest[:10])
#tempTrain.columns = ['feature', 'label']
#print(tempTrain.feature)
print(tempTrain[0][1])

tv


In [162]:
# Creating our feature vector X and our target vector y
#X_train = np(tempTrain.feature.tolist())

In [349]:
# Creating our feature matrix and target vector for the training and test set
X_train = []
y_train = []
for obj in tempTrain:
    #print(obj)
    X_train.append(obj[0])
    y_train.append(obj[1])

X_test = []
y_test = []
for obj in tempTest:
    print(obj)
    X_test.append(obj[0])
    y_test.append(obj[1])
#for obj in tempTrain:
#    print(obj)


[array([-4.98735687e+02,  7.29769669e+01, -7.63189840e+00,  2.15344963e+01,
        1.01189871e+01,  8.70023251e+00, -1.69920158e+01,  5.68006372e+00,
       -1.37542305e+01,  7.32722521e+00, -1.17701950e+01,  9.64634478e-01,
       -1.15434036e+01,  6.78404197e-02, -9.14878082e+00,  2.54599190e+00,
       -8.90166187e+00, -5.19858694e+00, -2.80854392e+00, -1.79771066e+00,
       -1.04671745e+01,  1.60132796e-02, -3.85705662e+00, -2.46077323e+00,
       -1.47261214e+00, -3.00321269e+00, -9.30453718e-01, -3.78773642e+00,
       -6.39797270e-01, -1.43419647e+00, -1.20994496e+00, -3.75134015e+00,
       -1.77742100e+00, -3.20055628e+00,  8.63250852e-01, -3.18253350e+00,
       -3.55127978e+00, -2.72077608e+00, -2.10028028e+00, -5.57580113e-01],
      dtype=float32), 'original']
[array([-5.8013715e+02,  3.1240751e+01, -9.1783686e+00,  2.6718287e+01,
        1.7296877e+01,  1.0443140e+01, -1.6949547e+01,  1.5332540e+01,
       -2.3931429e+01,  2.1849934e+01, -1.3356496e+01,  5.4458184e+00,


In [350]:
# Making separate target arrays for sklearn and keras frameworks, as they work on different target
print(y_train)
print(y_test)

# Creating the target vector for the classical machine learning algorithms in sklearn which expect integer arrays as an empty list
y_train_cl = []
for label in y_train:
    if label == 'original':
        y_train_cl.append(0)
    elif label == 'desktop':
        y_train_cl.append(1)
    elif label == 'soundbox':
        y_train_cl.append(2)
    elif label == 'soundbar':
        y_train_cl.append(3)
    else:
        y_train_cl.append(4)
print(y_train_cl)

# Doing the same thing as above for the test vector
y_test_cl = []
for label in y_test:
    if label == 'original':
        y_test_cl.append(0)
    elif label == 'desktop':
        y_test_cl.append(1)
    elif label == 'soundbox':
        y_test_cl.append(2)
    elif label == 'soundbar':
        y_test_cl.append(3)
    else:
        y_test_cl.append(4)
print(y_test_cl)


['soundbox', 'original', 'soundbox', 'soundbox', 'original', 'soundbox', 'original', 'desktop', 'tv', 'desktop', 'desktop', 'soundbar', 'tv', 'soundbar', 'tv', 'soundbar', 'tv', 'soundbox', 'desktop', 'soundbox', 'desktop', 'desktop', 'tv', 'desktop', 'soundbox', 'soundbox', 'soundbar', 'original', 'tv', 'original', 'tv', 'original', 'original', 'desktop', 'desktop', 'original', 'soundbar', 'tv', 'soundbar', 'tv', 'original', 'tv', 'soundbox', 'soundbox', 'soundbar', 'soundbar', 'tv', 'desktop', 'desktop', 'soundbar', 'original', 'original', 'soundbox', 'original', 'soundbar', 'soundbox', 'tv', 'original', 'tv', 'original', 'soundbox', 'original', 'soundbox', 'soundbar', 'original', 'soundbox', 'tv', 'soundbox', 'soundbox', 'desktop', 'soundbar', 'tv', 'soundbar', 'desktop', 'soundbar', 'soundbox', 'soundbox', 'soundbar', 'original', 'soundbar', 'soundbox', 'soundbox', 'tv', 'desktop', 'tv', 'tv', 'soundbar', 'soundbox', 'soundbox', 'soundbar', 'soundbox', 'tv', 'tv', 'desktop', 'sound

In [351]:
#print(X_train[1])
#print(y_train)

#X_train = np.array(X_train)
#y_train = np.array(y_train)
#
#X_test = np.array(X_test)
#y_test = np.array(y_test)
#
#print(X_train.shape)
#print(X_test.shape)

# Making separate target arrays for sklearn and keras frameworks, as they work on different target

# Hot encoding y for keras
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))

y_test = np_utils.to_categorical(lb.fit_transform(y_test))
print(y_train[0])
print(y_test[0])
print(y_train)

[0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0.]
[[0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0

In [352]:
X_train = np.array(X_train)
#y_train = np.array(y_train)

X_test = np.array(X_test)
#y_test = np.array(y_test)

print(X_train.shape)
print(X_test.shape)

# Hot encoding y
#lb = LabelEncoder()
#y_train = np_utils.to_categorical(lb.fit_transform(y_train))
#
#y_test = np_utils.to_categorical(lb.fit_transform(y_test))

(175, 40)
(75, 40)


In [342]:
# Standardizing the features
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

print(X_train[:3])
print(y_train[:5])

[[-0.02216797 -0.65377766 -1.1283584  -0.08740792  0.7500985  -0.5173679
  -0.33471087 -0.81130475 -0.07260235 -1.9200822   2.5425377  -1.3213586
  -0.2900435   0.33859482  1.1149688  -1.8887362   1.6214136  -0.21470794
   0.6123977  -1.0631332   0.4353835  -1.7965643  -0.077767   -1.0482727
   1.6157397   0.35603774  0.73579544 -0.50742984  1.9227425   0.86047953
   0.78126884  1.0224851   0.7210926   0.4103991   1.379354   -0.11019032
  -0.10054046  0.65645504  1.3119762  -0.03880136]
 [ 1.6182622   1.5776995   0.43024495  1.4559005   0.88490546  0.40507892
   0.12554614 -0.48783082  0.8673117   0.3328651  -1.2951367  -0.7688018
   0.47917658 -0.9576071   0.55708456  0.28785235  0.09618106 -0.93330556
  -0.33103362 -0.29312798  0.53025454  0.18935956  0.11856648 -0.4153535
  -0.14808135 -1.1725885   0.03496121 -1.0538017  -0.70979184 -2.7350361
   0.68006057 -0.3046199   0.6582387  -0.95868033 -2.2060194  -0.8928952
   0.71287024  0.13415752  0.10192142 -0.53524965]
 [ 0.27742356 -0.

In [367]:
y_test_cl = np.array(y_test_cl)

##START IMPLEMENTATION OF MODELS
linear = svm.SVC(kernel='linear', C=1, decision_function_shape='ovo').fit(X_train, y_train_cl)
linear_pred = linear.predict(X_test)
accuracy_lin = linear.score(X_test, y_test_cl)
print('Accuracy Linear Kernel:', accuracy_lin)

Accuracy Linear Kernel: 1.0


In [368]:
print(linear_pred)

[0 3 4 2 3 2 0 2 1 3 4 2 3 3 3 2 4 4 4 0 0 0 4 0 3 4 0 1 2 1 0 3 3 0 3 3 0
 4 4 1 3 1 0 4 4 4 4 2 3 0 1 3 1 1 1 0 0 1 3 4 3 0 2 4 1 0 4 3 3 2 4 2 0 1
 1]


In [369]:
print(y_test_cl)

[0 3 4 2 3 2 0 2 1 3 4 2 3 3 3 2 4 4 4 0 0 0 4 0 3 4 0 1 2 1 0 3 3 0 3 3 0
 4 4 1 3 1 0 4 4 4 4 2 3 0 1 3 1 1 1 0 0 1 3 4 3 0 2 4 1 0 4 3 3 2 4 2 0 1
 1]
